In [1]:
import datetime
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import lightgbm as lgb
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import label_binarize

import shap

%config InlineBackend.figure_format = 'retina'

In [2]:
from notebook import notebookapp
import urllib
import json
import ipykernel
from shutil import copy2

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None


def copy_current_nb(new_name):
    nb = notebook_path()
    if nb:
        new_path = os.path.join(os.path.dirname(nb), new_name+'.ipynb')
        copy2(nb, new_path)
    else:
        print("Current notebook path cannot be determined.")

In [3]:
df = pd.read_csv('Data/cover_type_engineered.csv')

In [4]:
df = df.loc[:, [col for col in df if not col.startswith('Cover_Type_')]]
X = df.drop(columns=['Cover_Type', 'Aspect_Sector'])
y = df['Cover_Type'] - 1

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")

warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.api.types")

# Assuming X and y are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

# Define the LightGBM classifier
estimator = lgb.LGBMClassifier(objective='multiclass', num_class=len(np.unique(y)))

# Define hyperparameters for tuning LightGBM
hyperparameters = {
    'num_leaves': stats.randint(30, 150),
    'min_child_samples': stats.randint(20, 500),
    'min_child_weight': stats.uniform(1e-5, 1e-2),
    'subsample': stats.uniform(0.5, 0.5),  
    'colsample_bytree': stats.uniform(0.5, 0.5),
    'reg_alpha': stats.uniform(1e-5, 1),
    'reg_lambda': stats.uniform(1e-5, 1),
    'learning_rate': stats.uniform(0.01, 0.3),
    'n_estimators': stats.randint(50, 500),
}

random_search = RandomizedSearchCV(estimator,
                                   param_distributions=hyperparameters,
                                   scoring='accuracy',  # Choose appropriate metric
                                   return_train_score=True,
                                   n_iter=500,
                                   cv=5,
                                   verbose=10,
                                   n_jobs=-1)

# Fit the RandomizedSearchCV
try:
    random_search.fit(X_train, y_train)  # Assuming X_train and y_train are defined
    print("Best parameters found:", random_search.best_params_)
    print("Best score found:", random_search.best_score_)

    
    # Save results
    results_path = f"./tuning_results/tuning_lgbm/{timestamp}"
    if not os.path.exists(results_path):
        os.makedirs(results_path)
        
    # Saving cross-validation results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results_file = f"{timestamp}_results.csv"
    cv_results.to_csv(os.path.join(results_path, cv_results_file), index=False)
    
    # Save .ipynb
    copy_current_nb(os.path.join(results_path, 'Evaluation_Notebook'))
    
    # Save Model
    file_name = f"lgbm_{timestamp}.pkl"
    pickle.dump(random_search, open(os.path.join(results_path, file_name), "wb"))
    
    # random_search = pickle.load(open(file_name, "rb"))
    

except Exception as e:
    print(f"An error occurred during model optimization: {e}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 1/5; 1/500] START colsample_bytree=0.957931407341758, learning_rate=0.14798891436244344, min_child_samples=457, min_child_weight=0.008557504687892561, n_estimators=410, num_leaves=135, reg_alpha=0.593905100504062, reg_lambda=0.6546088499394438, subsample=0.8947327855022912
[CV 1/5; 1/500] END colsample_bytree=0.957931407341758, learning_rate=0.14798891436244344, min_child_samples=457, min_child_weight=0.008557504687892561, n_estimators=410, num_leaves=135, reg_alpha=0.593905100504062, reg_lambda=0.6546088499394438, subsample=0.8947327855022912;, score=(train=1.000, test=0.856) total time=   6.2s
[CV 1/5; 4/500] START colsample_bytree=0.6062056232493445, learning_rate=0.2974894547758782, min_child_samples=277, min_child_weight=0.008712765129695923, n_estimators=91, num_leaves=136, reg_alpha=0.3021074899194223, reg_lambda=0.8127735491539001, subsample=0.9501615916275031
[CV 1/5; 4/500] END colsample_bytree=0.606205623249

[CV 3/5; 2/500] START colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294
[CV 3/5; 2/500] END colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294;, score=(train=0.998, test=0.857) total time=   3.9s
[CV 4/5; 2/500] START colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294
[CV 4/5; 2/500] END colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_sampl

[CV 1/5; 2/500] START colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294
[CV 1/5; 2/500] END colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294;, score=(train=0.998, test=0.857) total time=   4.1s
[CV 1/5; 3/500] START colsample_bytree=0.6551286221305597, learning_rate=0.024850666743911676, min_child_samples=20, min_child_weight=0.00334658941417462, n_estimators=77, num_leaves=47, reg_alpha=0.4590196133500141, reg_lambda=0.6858679287504249, subsample=0.996998505485779
[CV 1/5; 3/500] END colsample_bytree=0.6551286221305597, learning_rate=0.024850666743911676, min_child_samples=

[CV 4/5; 1/500] START colsample_bytree=0.957931407341758, learning_rate=0.14798891436244344, min_child_samples=457, min_child_weight=0.008557504687892561, n_estimators=410, num_leaves=135, reg_alpha=0.593905100504062, reg_lambda=0.6546088499394438, subsample=0.8947327855022912
[CV 4/5; 1/500] END colsample_bytree=0.957931407341758, learning_rate=0.14798891436244344, min_child_samples=457, min_child_weight=0.008557504687892561, n_estimators=410, num_leaves=135, reg_alpha=0.593905100504062, reg_lambda=0.6546088499394438, subsample=0.8947327855022912;, score=(train=1.000, test=0.863) total time=   5.7s
[CV 3/5; 3/500] START colsample_bytree=0.6551286221305597, learning_rate=0.024850666743911676, min_child_samples=20, min_child_weight=0.00334658941417462, n_estimators=77, num_leaves=47, reg_alpha=0.4590196133500141, reg_lambda=0.6858679287504249, subsample=0.996998505485779
[CV 3/5; 3/500] END colsample_bytree=0.6551286221305597, learning_rate=0.024850666743911676, min_child_samples=20, mi

[CV 2/5; 2/500] START colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294
[CV 2/5; 2/500] END colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294;, score=(train=0.997, test=0.848) total time=   4.2s
[CV 5/5; 2/500] START colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_samples=438, min_child_weight=0.00043063308326785846, n_estimators=317, num_leaves=73, reg_alpha=0.6499012319862668, reg_lambda=0.37227264404505983, subsample=0.9335210168715294
[CV 5/5; 2/500] END colsample_bytree=0.7676967674304089, learning_rate=0.1197306578497946, min_child_sampl

...

[CV 3/5; 438/500] START colsample_bytree=0.6464753842174675, learning_rate=0.24406431306647092, min_child_samples=203, min_child_weight=0.007694530819080923, n_estimators=326, num_leaves=48, reg_alpha=0.08103170902055727, reg_lambda=0.4693393605769905, subsample=0.7059568470077098
[CV 3/5; 438/500] END colsample_bytree=0.6464753842174675, learning_rate=0.24406431306647092, min_child_samples=203, min_child_weight=0.007694530819080923, n_estimators=326, num_leaves=48, reg_alpha=0.08103170902055727, reg_lambda=0.4693393605769905, subsample=0.7059568470077098;, score=(train=1.000, test=0.866) total time=   6.6s
[CV 2/5; 440/500] START colsample_bytree=0.7740302470475009, learning_rate=0.1328879157800038, min_child_samples=25, min_child_weight=0.008219242229652847, n_estimators=103, num_leaves=131, reg_alpha=0.6676557469745911, reg_lambda=0.06706033509013735, subsample=0.7129126891085484
[CV 2/5; 440/500] END colsample_bytree=0.7740302470475009, learning_rate=0.1328879157800038, min_child_s

[CV 2/5; 412/500] START colsample_bytree=0.5189620935539867, learning_rate=0.12371337740832654, min_child_samples=198, min_child_weight=0.0023504857483445064, n_estimators=472, num_leaves=76, reg_alpha=0.03604923479475735, reg_lambda=0.13179810643530127, subsample=0.7257884441702598
[CV 2/5; 412/500] END colsample_bytree=0.5189620935539867, learning_rate=0.12371337740832654, min_child_samples=198, min_child_weight=0.0023504857483445064, n_estimators=472, num_leaves=76, reg_alpha=0.03604923479475735, reg_lambda=0.13179810643530127, subsample=0.7257884441702598;, score=(train=1.000, test=0.861) total time=  10.5s
[CV 5/5; 413/500] START colsample_bytree=0.952825802323453, learning_rate=0.2047461864276974, min_child_samples=488, min_child_weight=0.005825610828310002, n_estimators=476, num_leaves=76, reg_alpha=0.6020471874451925, reg_lambda=0.7439701890042659, subsample=0.8496231257955937
[CV 5/5; 413/500] END colsample_bytree=0.952825802323453, learning_rate=0.2047461864276974, min_child_

[CV 2/5; 409/500] START colsample_bytree=0.7367991964696902, learning_rate=0.30986152917288406, min_child_samples=78, min_child_weight=0.008242846129731965, n_estimators=65, num_leaves=142, reg_alpha=0.11184591284277794, reg_lambda=0.507572045059672, subsample=0.7451810749374781
[CV 2/5; 409/500] END colsample_bytree=0.7367991964696902, learning_rate=0.30986152917288406, min_child_samples=78, min_child_weight=0.008242846129731965, n_estimators=65, num_leaves=142, reg_alpha=0.11184591284277794, reg_lambda=0.507572045059672, subsample=0.7451810749374781;, score=(train=1.000, test=0.856) total time=   2.1s
[CV 3/5; 410/500] START colsample_bytree=0.6883048230307229, learning_rate=0.20425082648536308, min_child_samples=286, min_child_weight=0.0005038343507018628, n_estimators=98, num_leaves=84, reg_alpha=0.4816579431568359, reg_lambda=0.07404449530496819, subsample=0.7015277266800248
[CV 3/5; 410/500] END colsample_bytree=0.6883048230307229, learning_rate=0.20425082648536308, min_child_sam

[CV 3/5; 416/500] START colsample_bytree=0.8574923242491597, learning_rate=0.2846661594905912, min_child_samples=148, min_child_weight=0.004251044358753849, n_estimators=279, num_leaves=77, reg_alpha=0.08607645524048928, reg_lambda=0.17329703571477723, subsample=0.9207966105105452
[CV 3/5; 416/500] END colsample_bytree=0.8574923242491597, learning_rate=0.2846661594905912, min_child_samples=148, min_child_weight=0.004251044358753849, n_estimators=279, num_leaves=77, reg_alpha=0.08607645524048928, reg_lambda=0.17329703571477723, subsample=0.9207966105105452;, score=(train=1.000, test=0.864) total time=   4.4s
[CV 3/5; 417/500] START colsample_bytree=0.5234807496209517, learning_rate=0.023301328915272992, min_child_samples=432, min_child_weight=0.0009630232924216154, n_estimators=479, num_leaves=97, reg_alpha=0.2102162770626928, reg_lambda=0.11753874687609504, subsample=0.6490970992648997
[CV 3/5; 417/500] END colsample_bytree=0.5234807496209517, learning_rate=0.023301328915272992, min_ch

[CV 4/5; 416/500] START colsample_bytree=0.8574923242491597, learning_rate=0.2846661594905912, min_child_samples=148, min_child_weight=0.004251044358753849, n_estimators=279, num_leaves=77, reg_alpha=0.08607645524048928, reg_lambda=0.17329703571477723, subsample=0.9207966105105452
[CV 4/5; 416/500] END colsample_bytree=0.8574923242491597, learning_rate=0.2846661594905912, min_child_samples=148, min_child_weight=0.004251044358753849, n_estimators=279, num_leaves=77, reg_alpha=0.08607645524048928, reg_lambda=0.17329703571477723, subsample=0.9207966105105452;, score=(train=1.000, test=0.866) total time=   4.5s
[CV 2/5; 418/500] START colsample_bytree=0.6449905690342621, learning_rate=0.13807251401223045, min_child_samples=432, min_child_weight=0.009861625481783062, n_estimators=267, num_leaves=80, reg_alpha=0.9928037819733815, reg_lambda=0.1974075238142588, subsample=0.5762064939885562
[CV 2/5; 418/500] END colsample_bytree=0.6449905690342621, learning_rate=0.13807251401223045, min_child_

In [ ]:
cv_results.head()

In [ ]:
results_path = f"./tuning_results/tuning_lgbm/{timestamp}/Assets"
if not os.path.exists(results_path):
    os.makedirs(results_path)

# CV Evaluation

In [ ]:
cv_results.columns

In [ ]:
cv_results.sort_values(by='rank_test_score', ascending=True).head(5)

In [ ]:
sorted_cv = cv_results.sort_values(by='rank_test_score', ascending=True)

# Train vs Test Comparison

In [ ]:
plt.figure(figsize=(16, 6))    

plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_train_score'], label="Train Score")
plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_test_score'], label="Test Score")

plt.grid()
plt.xlabel('Sorted Test Rank')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy by Final Rank (top 15)')
plt.legend(loc='best')

filename = "test_train_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Test Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_test_score', 'split1_test_score', 'split2_test_score',
   'split3_test_score', 'split4_test_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "test_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Train Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_train_score', 'split1_train_score', 'split2_train_score',
   'split3_train_score', 'split4_train_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "train_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
max_params = cv_results.loc[cv_results['rank_test_score'] == 1]
best_params = max_params.params.values[0]

In [ ]:
print(f"Mean Train set, Accuracy = {max_params['mean_train_score'].values[0]:.2f}")
print(f"Mean Test  set, Accuracy = {max_params['mean_test_score'].values[0]:.2f}")

In [ ]:
random_search = pickle.load(open(os.path.join(f"./tuning_results/tuning_lgbm/{timestamp}", file_name), "rb"))
model = random_search.best_estimator_

#model = LGBMClassifier(**best_params)
#model.fit(X_train, y_train)

y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

In [ ]:
print(f"Train set, Accuracy = {accuracy_score(y_train, y_train_prediction):.2f}")
print(f"Test set, Accuracy = {accuracy_score(y_test, y_test_prediction):.2f}")

In [ ]:
ind = np.argpartition(model.feature_importances_, -20)[-20:]

features = X.columns[ind]
importance = model.feature_importances_[ind]

fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(ind)), importance, align='center')
plt.yticks(range(len(ind)), features)
plt.title('Feature Importance LGBM')
plt.grid()

filename = "feature_importance.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()

In [ ]:
# TEST
for col in ['param_n_estimators', 'param_learning_rate']:
    
    plt.figure(figsize=(16, 6))    

    m, b = np.polyfit(list(sorted_cv['rank_test_score'].values), list(sorted_cv[col].values), 1)
    plt.plot(sorted_cv['rank_test_score'], m * sorted_cv['rank_test_score'] + b, c='r', label="Regression Line")
    plt.plot(sorted_cv['rank_test_score'], sorted_cv[col], label=f"{col} Values")
    
    plt.grid()
    plt.xlabel('Hyperparameter Values')
    plt.ylabel('Parameter Value')
    plt.title(col)
    plt.legend(loc='best')

    
    filename = f"{col}_by_rank.png"
    plt.savefig(os.path.join(results_path, filename))
                  
    plt.show()


# Hyperparameter Evaluation

In [ ]:

def plot_parameters(x_values, title):
    
    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax2 = ax1.twinx()

    ax1.scatter(x_values, cv_results['mean_test_score'], label='mean_test_score', c='b')
    ax2.scatter(x_values, cv_results['std_test_score'], label='std_test_score', c='r')

    m, b = np.polyfit(list(x_values.values), list(cv_results['mean_test_score'].values), 1)
    ax1.plot(x_values, m * x_values + b, c='b')

    m, b = np.polyfit(list(x_values.values), list(cv_results['std_test_score'].values), 1)
    ax2.plot(x_values, m * x_values + b, c='r')
    
    ax1.set_title(title)
    ax1.set_xlabel('Parameter Value')
    ax1.set_ylabel('Mean Test Score')
    ax2.set_ylabel('Standard Deviation of Test Score')
    ax1.grid(True)
    
    
    # Combine the legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    filename = f"{title}_test_score.png"
    plt.savefig(os.path.join(results_path, filename))
            
    plt.show()



In [ ]:

for param in ['param_n_estimators', 'param_learning_rate']:
    x_values = cv_results[param]
    plot_parameters(x_values, param)

# Plotting Evaluation Metrics (Precision, Recall, F1-Score, AUC-ROC):


In [ ]:

# For multiclass classification, you need to binarize the labels
y_true_bin = label_binarize(y_test, classes=np.unique(y_test))
y_score_bin = label_binarize(y_test_prediction, classes=np.unique(y_test_prediction))

auc_roc = roc_auc_score(y_true_bin, y_score_bin, average='macro')

# Plot Precision-Recall curve for each class
precision = dict()
recall = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(recall[i], precision[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('Recall')
plt.ylabel('True Positive Rate / Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')

filename = "precision_recall.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Plot AUC-ROC curve for each class
fpr = dict()
tpr = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(fpr[i], tpr[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate / Precision')
plt.title('ROC Curve')
plt.legend(loc='best')
            
filename = "roc_curve.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Partial Dependence

In [ ]:
# potentially iterate over features (and relation ie 0 to 1)

In [ ]:

features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 1, num=5)}

pd_results = partial_dependence(
    model, X, features=1, kind="average", grid_resolution=5)

display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot(pdp_lim={1: (-1.38, 0.66)})

plt.grid()
plt.xlabel('Feature Value')
plt.ylabel('Partial Dependence') 
plt.title('Partial Dependence')

filename = "partial_dependence.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Confusion Matrix

In [ ]:

class_names = np.unique(y)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X_test,
        y_test,
        display_labels=class_names + 1,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    png_name = title.lower().replace(" ", "_")
    filename = f"{png_name}.png"
    plt.savefig(os.path.join(results_path, filename))

plt.show()

# Shap Values

In [ ]:
explainer = shap.TreeExplainer(model)
explanation = explainer.shap_values(X_test, check_additivity=False)


In [ ]:
shap.summary_plot(explanation, X_test, plot_type="bar", show=False)

filename = f"shap_summary.png"
plt.savefig(os.path.join(results_path, filename))
plt.close()  

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.
